In [ ]:
from Bio.Seq import Seq
from Bio.Data import CodonTable
from Bio import SeqIO

In [ ]:
def gerar_quadros_leitura(seq_nucleotideos, tabela_traducao):
    """
    Gera os 6 quadros de leitura (3 diretos e 3 reversos) para uma sequência de nucleotídeos.

    Args:
    seq_nucleotideos (str): Sequência de nucleotídeos.
    tabela_traducao (str): Nome da tabela de tradução a ser utilizada.

    Retorna:
    list: Lista com as 6 sequências de quadros de leitura traduzidas possíveis.
    """
    seq = Seq(seq_nucleotideos)

    # 3 quadros de leitura diretos
    quadros_diretos = [traduzir_nucleotideos_para_AA(seq[i:], tabela_traducao) for i in range(3)]
    
    # Para os 3 quadros de leitura reversos
    seq_inversa = str(seq[::-1])  # Inverte a sequência de nucleotídeos
    quadros_reversos = [traduzir_nucleotideos_para_AA(Seq(seq_inversa[i:]), tabela_traducao) for i in range(3)]

    # Combina os quadros diretos e reversos
    return quadros_diretos + quadros_reversos

In [ ]:
def traduzir_nucleotideos_para_AA(seq_nucleotideos, tabela_traducao):
    """
    Transforma uma sequência de nucleotídeos em uma sequência de aminoácidos.
    Identifica códons de parada e usa a tabela de tradução.

    Args:
    seq_nucleotideos (Bio.Seq.Seq): Sequência de nucleotídeos.
    tabela_traducao (str): Tabela de tradução espécie-específica.

    Retorna:
    str: Sequência de aminoácidos correspondente.
    """
    # Verifica se a tabela de tradução fornecida está disponível
    if tabela_traducao not in CodonTable.unambiguous_dna_by_name:
        raise ValueError(f"Tabela de tradução {tabela_traducao} não encontrada.")

    # Usa a tabela de tradução especificada
    tabela = CodonTable.unambiguous_dna_by_name[tabela_traducao]

    # Traduz a sequência de nucleotídeos para aminoácidos, considerando códons de parada
    aa_sequence = seq_nucleotideos.translate(table=tabela, to_stop=False)

    return str(aa_sequence)

In [ ]:
def escolher_melhor_quadro(quadros):
    """
    Escolhe o melhor quadro de leitura baseado na presença do códon de início (Metionina)
    e se mais de um tiver o códon de inicio, escolhe a maior sequência entre ele e o códon de parada (*).

    Args:
    quadros (list): Lista de quadros de leitura.

    Retorna:
    tupla: Uma tupla contendo a maior sequência de aminoácidos e o índice do quadro ao qual pertence.
    """
    maior_sequencia = ''
    indice_melhor_quadro = -1

    for i, quadro in enumerate(quadros):
        # Quebra o quadro em sub-sequências, separando-as com base no códon de parada '*'
        sub_sequencias = quadro.split('*')

        # Procura as sub-sequências que começam com 'M' (códon de início)
        for sub_seq in sub_sequencias:
            if 'M' in sub_seq:
                # Extrai a parte da sequência que começa com 'M'
                inicio_metionina = sub_seq[sub_seq.index('M'):]
                
                # Verifica se esta sub-sequência é maior que a maior encontrada até agora
                if len(inicio_metionina) > len(maior_sequencia):
                    maior_sequencia = inicio_metionina
                    indice_melhor_quadro = i

    # Retorna a maior sequência e o índice do quadro ao qual pertence
    return maior_sequencia, indice_melhor_quadro if maior_sequencia else None

In [ ]:
def carregar_sequencia_fasta(arquivo_fasta):
    """
    Carrega uma sequência de nucleotídeos a partir de um arquivo FASTA.

    Args:
    arquivo_fasta (str): Caminho para o arquivo FASTA.

    Retorna:
    Bio.Seq.Seq: Sequência de nucleotídeos.
    """
    # Carregar a sequência usando o SeqIO
    with open(arquivo_fasta, "r") as file:
        for record in SeqIO.parse(file, "fasta"):
            return record.seq 

In [ ]:
def identificar_tabela_traducao(fasta_header):
    """
    Identifica a tabela de tradução apropriada para um arquivo FASTA
    baseado no cabeçalho/metadados.

    Args:
    fasta_header (str): Cabeçalho da sequência FASTA com informações sobre o organismo.

    Retorna:
    str: Nome da tabela de tradução apropriada.
    """

    # Exemplos de indentificação do organismo
    if "mitochondrion" in fasta_header:
        if "vertebrate" in fasta_header:
            return "Vertebrate Mitochondrial"
        elif "invertebrate" in fasta_header:
            return "Invertebrate Mitochondrial"
        elif "yeast" in fasta_header:
            return "Yeast Mitochondrial"
    elif "bacteria" in fasta_header or "archaea" in fasta_header:
        return "Bacterial"

    # Caso nenhuma regra se aplique, retornar a tabela padrão
    return "Standard"

In [ ]:
arquivo_fasta = input("Qual o caminho do arquivo .fasta?")
sequencia = carregar_sequencia_fasta(arquivo_fasta)

# Mostra a tabela usada com base no cabeçalho
tipo = identificar_tabela_traducao(arquivo_fasta)
print(f'Tabela usada: {tipo}')

In [ ]:
quadros = gerar_quadros_leitura(sequencia, tipo)

# Mostra os 6 quadros de leitura
for i, quadro in enumerate(quadros):
    print(f"Quadro {i+1}: {quadro}")

# Escolhe o melhor quadro de leitura
melhor_sequencia, indice_quadro = escolher_melhor_quadro(quadros)
print(f"\nMelhor quadro de leitura: {indice_quadro+1}\n")
print(f"\nMelhor sequencia: {melhor_sequencia}")

In [ ]:
from Bio.Data import CodonTable
from itertools import product

In [ ]:
def traduzir_AA_para_nucleotideos(seq_AA, tabela_traducao):
    """
    Traduz uma sequência de aminoácidos de volta para nucleotídeos, considerando todas as possibilidades de códons.

    Args:
    seq_AA (str): Sequência de aminoácidos.
    tabela_traducao (str): Tabela de tradução a ser usada.

    Retorna:
    str: Uma sequência de nucleotídeos correspondente e a contagem de mutantes para cada códon.
    """
    # Acessa a tabela de tradução
    tabela = CodonTable.unambiguous_dna_by_name[tabela_traducao]
    
    # Dicionário para armazenar os códons correspondentes a cada aminoácido
    codons_por_aminoacido = {}
    
    for codon, amino_acid in tabela.forward_table.items():
        if amino_acid not in codons_por_aminoacido:
            codons_por_aminoacido[amino_acid] = []
        codons_por_aminoacido[amino_acid].append(codon)

    # Monta uma lista de listas de códons para cada aminoácido na sequência
    codon_combinacoes = []
    for aa in seq_AA:
        if aa in codons_por_aminoacido:
            codon_combinacoes.append(codons_por_aminoacido[aa])
        elif aa == '*':
            codon_combinacoes.append(['TAA', 'TAG', 'TGA']) 
        else:
            raise ValueError(f"Aminoácido '{aa}' não encontrado na tabela de tradução.")

    # Seleciona uma combinação de códons (apenas a primeira)
    nucleotideos_selecionados = []
    quantidades_mutantes = []

    for codons in codon_combinacoes:
        # Pega o primeiro códon da lista
        nucleotideos_selecionados.append(codons[0])
        # Conta quantos códons existem para esse aminoácido
        quantidades_mutantes.append(len(codons))

    # Junta os nucleotídeos e formata a saída
    nucleotidos_final = ''.join(nucleotideos_selecionados)
    mutantes_formatados = ', '.join([f"{nuc} ({quant})" for nuc, quant in zip(nucleotideos_selecionados, quantidades_mutantes)])

    return nucleotidos_final, mutantes_formatados

In [ ]:
# Obtém a sequência de nucleotideos da melhor sequencia
nucleotideos_resultantes, mutantes_info = traduzir_AA_para_nucleotideos(melhor_sequencia, tipo)

# Mostrando o resultado
print(f"Nucleotídeos traduzidos: {nucleotideos_resultantes}")
print(f"Informações sobre os mutantes: {mutantes_info}")